# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [43]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [44]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [45]:
# Your answer here
#it has 178 observation rows and 14 columns
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  class

_(ii)_ How many variables (columns) does the dataset contain?

In [46]:
# Your answer here

# The dataset contains 14 columns
print("Total number of columns: ", wine_df.shape[1])

Total number of columns:  14


_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [47]:
# Your answer here
#class is an Integer type with 3 uniquie values

wine_df['class'].unique()

array([0, 1, 2])


_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [48]:
# Your answer here
#we have 13 predictor variable
print("number of predictor variables: ", wine_df.shape[1] - 1)

number of predictor variables:  13


You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [49]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

> Your answer here...

it is important because predictors may be measured on different scales which can distort the model. Standardization therefore puts everything on the same scale, so the model treats each variable fairly and one scale is not overpowering the distance calculation therefore misleading results

(ii) Why did we elect not to standard our response variable `Class`?

> Your answer here...
because the y (response variable) is what we are trying to predict so it would be unnecessary to standardize it if it is not a predictor.

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

> Your answer here...
Setting a random seed is important because it makes the randomly generated operations repeatable so that we get the same result for replicability and consistency purposes. the particular seed value is not important but needs to be consistent.

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [58]:
# set a seed for reproducibility
np.random.seed(123)

# split the data into a training and testing set. hint: use train_test_split !

Wine_train, Wine_test = train_test_split(predictors_standardized, train_size=0.75, shuffle = True)    

# Your code here ...
Wine_train.info()


<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, 28 to 109
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       133 non-null    float64
 1   malic_acid                    133 non-null    float64
 2   ash                           133 non-null    float64
 3   alcalinity_of_ash             133 non-null    float64
 4   magnesium                     133 non-null    float64
 5   total_phenols                 133 non-null    float64
 6   flavanoids                    133 non-null    float64
 7   nonflavanoid_phenols          133 non-null    float64
 8   proanthocyanins               133 non-null    float64
 9   color_intensity               133 non-null    float64
 10  hue                           133 non-null    float64
 11  od280/od315_of_diluted_wines  133 non-null    float64
 12  proline                       133 non-null    float64
dtypes: float6

#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [59]:
# Your code here...
knn = KNeighborsClassifier(n_neighbors=5)
X = Wine_train
y = wine_df.loc[X.index, 'class'] #Target variable from original df
knn.fit(X, y)   

returned_dictionary = cross_validate(estimator = knn, X = X , y = y, cv=10)

cv_10_df = pd.DataFrame(returned_dictionary)

cv_10_metrics = cv_10_df.agg(['mean', 'sem'])
cv_10_metrics

,fit_time,score_time,test_score
mean,0.005223,0.009232,0.961538
sem,0.000437,0.002142,0.023640


#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [52]:
# Your code here...
parameter_grid = {'n_neighbors':range(1, 51, 5)}
wine_tune_grid = GridSearchCV(
    estimator=knn,
    param_grid=parameter_grid,
    cv=10
)
wine_tune_grid.fit(X, y)   
accuracy_grid = pd.DataFrame(wine_tune_grid.cv_results_)
accuracy_grid  


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003124,0.001014,0.006470,0.002332,1,{'n_neighbors': 1},1.0,0.928571,0.928571,1.0,0.923077,1.000000,0.923077,1.0,0.769231,0.923077,0.939560,0.066844,10
1,0.003038,0.001104,0.004375,0.000608,6,{'n_neighbors': 6},1.0,1.000000,1.000000,1.0,0.923077,1.000000,1.000000,1.0,0.769231,1.000000,0.969231,0.070501,7
2,0.003366,0.001499,0.007471,0.005767,11,{'n_neighbors': 11},1.0,1.000000,1.000000,1.0,0.923077,1.000000,1.000000,1.0,0.846154,0.923077,0.969231,0.051025,7
3,0.002813,0.000855,0.004582,0.000824,16,{'n_neighbors': 16},1.0,1.000000,1.000000,1.0,0.923077,1.000000,1.000000,1.0,0.923077,0.923077,0.976923,0.035251,3
4,0.003626,0.001163,0.004310,0.000722,21,{'n_neighbors': 21},1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,0.923077,0.923077,0.984615,0.030769,1
5,0.002735,0.000957,0.004775,0.000657,26,{'n_neighbors': 26},1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,0.923077,0.923077,0.984615,0.030769,1
6,0.003787,0.001254,0.004663,0.001294,31,{'n_neighbors': 31},1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,0.846154,0.923077,0.976923,0.049255,3
7,0.003069,0.000868,0.004774,0.001561,36,{'n_neighbors': 36},1.0,0.928571,1.000000,1.0,1.000000,1.000000,1.000000,1.0,0.923077,0.846154,0.969780,0.050552,5
8,0.005519,0.003490,0.003935,0.000563,41,{'n_neighbors': 41},1.0,0.928571,1.000000,1.0,1.000000,0.923077,1.000000,1.0,0.923077,0.923077,0.969780,0.037042,5
9,0.002906,0.000528,0.004608,0.000944,46,{'n_neighbors': 46},1.0,0.857143,1.000000,1.0,0.923077,0.923077,1.000000,1.0,0.923077,0.846154,0.947253,0.058314,9


In [53]:

wine_tune_grid.best_params_ 


{'n_neighbors': 21}

In [54]:
knn = KNeighborsClassifier(n_neighbors=wine_tune_grid.best_params_['n_neighbors'])
X = Wine_train
y = wine_df.loc[X.index, 'class']
knn.fit(X, y)

,n_neighbors,21
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [55]:
Wine_test["predicted"] = knn.predict(Wine_test)
Wine_test

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,predicted
138,0.604516,1.125357,-0.645229,0.001518,-0.824415,-1.081754,-1.555415,1.757779,-1.245555,0.277671,-0.646911,-1.118210,-0.531471,2
60,-0.828391,-1.109824,-0.316249,-1.049479,0.088358,-0.392751,-0.942995,2.160669,-2.069034,-0.773474,1.283518,-1.330077,-0.213021,1
137,-0.581338,2.848870,0.999674,1.653086,-0.262708,-0.809357,-1.434939,2.160669,-0.860096,-0.025128,-0.603037,-1.301828,-0.738463,2
77,-1.433671,0.496993,-0.499016,-0.448909,0.860705,-0.921521,-0.712083,0.549108,-1.122909,-1.041667,0.011190,-0.129500,-0.786230,1
90,-1.137207,-0.454530,-0.170035,-0.298767,-1.315908,-1.113800,-0.531369,1.274310,0.086029,-1.149810,0.537671,-0.482611,-0.849920,1
159,0.592164,-0.598156,0.999674,0.902373,-0.754202,0.488531,-0.932956,1.274310,1.224884,2.894719,-1.699872,-1.174708,-0.404091,2
41,0.505695,1.349773,-0.901103,-0.208681,-0.683988,0.248181,0.653312,-0.740141,-0.194304,-0.336578,-0.208177,0.548472,0.917474,0
136,-0.927212,2.139716,0.634140,0.451946,-0.754202,-1.466313,-1.565455,1.354888,-1.385721,-0.522583,-0.910151,-1.895054,-0.085641,2
166,0.555106,1.224100,0.853460,1.052516,0.790492,-0.953567,-1.113670,0.549108,-0.229346,2.431870,-0.471417,-1.485445,-0.165254,2
93,-0.877801,0.443133,-0.535569,-0.448909,-0.824415,0.248181,0.221606,-0.901297,0.699259,-1.257952,0.844785,0.972205,-1.454974,1


# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `11:59 PM - 11/24/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/LCR/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#dc2-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
